In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
url="http://data.chicagojustice.org/articles/"

In [3]:
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

In [4]:
# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

In [5]:
# Step 3: Analyze the HTML tag, where your content lives
# Create a data dictionary to store the data.
data = {}
#Get the table having the class wikitable
article = soup.find_all("table", attrs={"id": "articleTable"})
print("Number of tables on site: ",len(article))

Number of tables on site:  1


In [6]:
article_data = article[0].find_all("tr")

In [7]:
head = article_data[0] # 0th item is the header row
body_rows = article_data[1:] # All other items becomes the rest of the rows

In [8]:
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)
print(headings)

['', 'Load Time', 'Load Date', 'Source', 'Title', 'Link', 'Snippet']


In [9]:
# Next is now to loop though the rest of the rows

#print(body_rows[0])
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [10]:
# We can now use the data on all_rowsa and headings to make a table
# all_rows becomes our data and headings the column names
df = pd.DataFrame(data=all_rows,columns=headings)
df.head()

,,Load Time,Load Date,Source,Title,Link,Snippet
0,View Article,10:01 a.m.,February 7 2021,WGN TV,Sunday Brunch: Chicken Wings and Buffalo Sauce,https://wgntv.com/news/wgn-weekend-morning-new...,Chef Frankie Celenza joins WGN Weekend Morning...
1,View Article,10:01 a.m.,February 7 2021,WGN TV,Officials plead: Don’t let Super Bowl become s...,https://wgntv.com/news/coronavirus/officials-p...,Kansas City Chiefs superfan Ty Rowton hugged s...
2,View Article,10:01 a.m.,February 7 2021,WGN TV,AstraZeneca vaccine being tweaked to fight Sou...,https://wgntv.com/news/coronavirus/astrazeneca...,LONDON (AP) — Developers of the Oxford-AstraZe...
3,View Article,10:01 a.m.,February 7 2021,WGN TV,Next stop Mars: 3 spacecraft arriving in quick...,https://wgntv.com/news/next-stop-mars-3-spacec...,CAPE CANAVERAL Fla. (AP) — After hurtling hund...
4,View Article,10:01 a.m.,February 7 2021,NBC Local,Chicago Weather: Bitterly Cold Temperatures Wi...,https://www.nbcchicago.com/weather/chicago-wea...,A wind chill advisory will be in effect until ...
